# Multi-table Datasets - ENRON Archive

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

> Answer:
There are 3 tables:
- MessageBase
- RecipientBase
- EmployeeBase

In [1]:
import sqlite3
conn = sqlite3.connect('../../assets/datasets/enron.db') 
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()


[(u'MessageBase',), (u'RecipientBase',), (u'EmployeeBase',)]

Query the `sqlite_master` table to retrieve the schema of the `EmployeeBase` table.

1. What fields are there?
1. What's the type of each of them?

In [50]:
employee_columns = c.execute("PRAGMA table_info(EmployeeBase)").fetchall()
employee_column_types = [x[2] for x in employee_columns]
employee_columns = [x[1] for x in employee_columns]

message_columns = c.execute("PRAGMA table_info(MessageBase)").fetchall()
message_column_types = [x[2] for x in message_columns]
message_columns = [x[1] for x in message_columns]

recipient_columns = c.execute("PRAGMA table_info(RecipientBase)").fetchall()
recipient_column_types = [x[2] for x in recipient_columns]
recipient_columns = [x[1] for x in recipient_columns]

print "Employee base columns: %s" % employee_columns
print "Employee base column types: %s" % employee_column_types
print "Message base columns: %s" % message_columns
print "Message base column types: %s" % message_column_types
print "recipient base columns: %s" % recipient_columns
print "recipient base column types: %s" % recipient_column_types


Employee base columns: [u'eid', u'name', u'department', u'longdepartment', u'title', u'gender', u'seniority']
Employee base column types: [u'INTEGER', u'TEXT', u'TEXT', u'TEXT', u'TEXT', u'TEXT', u'TEXT']
Message base columns: [u'mid', u'filename', u'unix_time', u'subject', u'from_eid']
Message base column types: [u'INTEGER', u'TEXT', u'INTEGER', u'TEXT', u'INTEGER']
recipient base columns: [u'mid', u'rno', u'to_eid']
recipient base column types: [u'INTEGER', u'INTEGER', u'INTEGER']


1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

In [24]:
c.execute("SELECT * FROM EmployeeBase LIMIT 5;").fetchall()
c.execute("SELECT * FROM MessageBase LIMIT 5;").fetchall()
c.execute("SELECT * FROM RecipientBase LIMIT 5;").fetchall()

[(1, 1, 59), (2, 1, 15), (3, 1, 15), (4, 1, 109), (4, 2, 49)]

Import each of the 3 tables to a Pandas Dataframes

In [269]:
import pandas as pd
employee_base = pd.DataFrame(c.execute("SELECT * FROM EmployeeBase").fetchall(), columns = employee_columns)
message_base = pd.DataFrame(c.execute("SELECT * FROM MessageBase").fetchall(), columns = message_columns)
recipient_base = pd.DataFrame(c.execute("SELECT * FROM RecipientBase").fetchall(), columns = recipient_columns)
employee_base.head()

,eid,name,department,longdepartment,title,gender,seniority
0,1,John Arnold,Forestry,ENA Gas Financial,VP Trading,Male,Senior
1,2,Harry Arora,Forestry,ENA East Power,VP Trading,Male,Senior
2,3,Robert Badeer,Forestry,ENA West Power,Mgr Trading,Male,Junior
3,4,Susan Bailey,Legal,ENA Legal,Specialist Legal,Female,Junior
4,5,Eric Bass,Forestry,ENA Gas Texas,Trader,Male,Junior


In [55]:
message_base.head()

,mid,filename,unix_time,subject,from_eid
0,1,taylor-m/sent/11,910930020,Cd$ CME letter,138
1,2,taylor-m/sent/17,911459940,Indemnification,138
2,3,taylor-m/sent/18,911463840,Re: Indemnification,138
3,4,taylor-m/sent/23,911874180,"Re: Coral Energy, L.P.",138
4,5,taylor-m/sent/27,912396120,Bankruptcy Code revisions,138


In [117]:
recipient_base.head(10)

,mid,rno,to_eid
0,1,1,59
1,2,1,15
2,3,1,15
3,4,1,109
4,4,2,49
5,4,3,120
6,4,4,59
7,5,1,45
8,5,2,53
9,6,1,113


## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company?
- How many messages are there in the database?
- Convert the timestamp column in the messages. When was the oldest message sent? And the newest?
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - How many messages have only one recepient?
    - How many messages have >= 5 recepients?
    - What's the highest number of recepients?
    - Who sent the message with the highest number of recepients?
- Plot the distribution of recepient numbers using Bokeh.

In [395]:
print "1. There are %s employees in the company" % len(employee_base['eid'])
print "2. There are %s messages inthe dataset" % len(message_base['filename'])
message_base['date_time'] = pd.to_datetime(message_base['unix_time'],unit='s')
print "3a. Newest: %s" % max(message_base['date_time'])
print "3b. Oldest: %s" % min(message_base['date_time'])

merged = message_base.merge(recipient_base, how = 'inner', on = 'mid')
x = pd.pivot_table(merged, values = ['rno'], index = ['mid'], aggfunc = len)
print "4a. Messages with one recipient: %s" % len(x[x['rno'] == 1])
print "4b. Messages with one recipient: %s" % len(x[x['rno'] >= 5])
print "4c. Highest: %s" % max(x.sort_values('rno', ascending = False)['rno'])

z = pd.pivot_table(merged, values = ['rno'], index = ['mid','from_eid'], aggfunc = len)
stuff = employee_base[employee_base['eid'] == z.sort_values('rno', ascending = False)['rno'].index[0][1]]['name']
print "4d. who sent it: %s" % stuff.values[0].encode('ascii','ignore')

1. There are 156 employees in the company
2. There are 21635 messages inthe dataset
3a. Newest: 2002-06-21 13:37:34
3b. Oldest: 1998-11-13 04:07:00
4a. Messages with one recipient: 14985
4b. Messages with one recipient: 1380
4c. Highest: 57
4d. who sent it: John J. Lavorato


Rescale to investigate the tail of the curve

In [217]:
from bokeh.charts import Histogram, show, output_notebook

p = Histogram(z, values = 'rno')
show(p)

## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)

In [346]:
temp = employee_base

print "%s males" % len(employee_base[(employee_base['gender'] == 'Male')])
print "%s females" % len(employee_base[(employee_base['gender'] == 'Female')])
pivot = pd.pivot_table(temp[['gender','department']], columns = ['gender'], index = ['department'], aggfunc = len)
print pivot

message_base['eid'] = message_base['from_eid']
mergy = message_base.merge(employee_base, how = 'inner', on = 'eid')

print "male messages: %s" % len(mergy[(mergy['gender'] == 'Male')])
print "female messages: %s" % len(mergy[(mergy['gender'] == 'Female')])
print "Avg male messages rounded: %s" % (12841/113)
print "Avg female messages rounded: %s" % (8794/43)
x = employee_base['seniority'].value_counts()
print "There are %s Juniors and %s Seniors" % (x[0],x[1])
print "Junior messages: %s" % len(mergy[(mergy['seniority'] == 'Junior')])
print "Senior messages: %s" % len(mergy[(mergy['seniority'] == 'Senior')])
print "Seniors send more. \n"
print "Messages sent by department: "
print mergy['department'].value_counts()
print "\n"
print "Top 3 senders and amounts"
print mergy['name'].value_counts()[:3]

113 males
43 females
gender      Female  Male
department              
Forestry        10    50
Legal           13    12
Other           20    51
male messages: 12841
female messages: 8794
Avg male messages rounded: 113
Avg female messages rounded: 204
There are 82 Juniors and 74 Seniors
Junior messages: 9196
Senior messages: 12439
Seniors send more. 

Messages sent by department: 
Legal       10396
Other        6852
Forestry     4387
Name: department, dtype: int64


Top 3 senders and amounts
Jeff Dasovich      1597
Tana Jones         1379
Sara Shackleton    1142
Name: name, dtype: int64


Answer the following questions regarding received messages:

- Who is receiving more emails? Men or Women?
- Who is receiving more emails? Juniors or Seniors?
- Which department is receiving more emails? How does that relate with the number of employees in the department?
- Who are the top 5 receivers of emails? (people who received the most emails)

In [384]:
recipient_base['eid'] = recipient_base['to_eid']
mergy2 = recipient_base.merge(employee_base, how = 'inner', on = 'eid')
print "received emails by gender, males receive more"
print mergy2['gender'].value_counts()
print "received emails by seniority, seniors receive more"
print mergy2['seniority'].value_counts()
print "received emails by department, legal receives more \n"
x = mergy2['department'].value_counts()
total = sum(x)

print "Legal recevies %s emails, %.2f percent of the total emails, and has %.2f percent of the employees." %(x[0],(x[0] / float(total))*100,60/float(156)*100)
print "Other recevies %s emails, %.2f percent of the total emails, and has %.2f percent of the employees." %(x[1],(x[1] / float(total))*100,25/float(156)*100)
print "Legal recevies %s emails, %.2f percent of the total emails, and has %.2f percent of the employees." %(x[2],(x[2] / float(total))*100,71/float(156)*100)
print "\n"
print "Top 5 receivers and amounts"
print mergy2['name'].value_counts()[:5]


received emails by gender, males receive more
Male      25549
Female    12839
Name: gender, dtype: int64
received emails by seniority, seniors receive more
Senior    23934
Junior    14454
Name: seniority, dtype: int64
received emails by department, legal receives more 

Legal recevies 16311 emails, 42.49 percent of the total emails, and has 38.46 percent of the employees.
Other recevies 13653 emails, 35.57 percent of the total emails, and has 16.03 percent of the employees.
Legal recevies 8424 emails, 21.94 percent of the total emails, and has 45.51 percent of the employees.


Top 5 receivers and amounts
James D. Steffes    1797
Richard Shapiro     1730
Mark E. Taylor      1477
Steven J. Kean      1290
Sara Shackleton     1173
Name: name, dtype: int64


Which employees sent the most 'mass' emails?

In [400]:
recipient_base[recipient_base['rno'] > 1].head()


,mid,rno,to_eid,eid
4,4,2,49,49
5,4,3,120,120
6,4,4,59,59
8,5,2,53,53
16,12,2,120,120


In [418]:
mass_email_merge = message_base.merge(recipient_base[recipient_base['rno'] > 1], how = 'inner', on = 'mid')
mass_email_merge['eid'] = mass_email_merge['to_eid']
mass_email_merge = employee_base.merge(mass_email_merge, how = 'inner', on = 'eid')

print "Top ten mass email (>2 recipients) senders"
print mass_email_merge['name'].value_counts().head(10)

Top ten mass email (>2 recipients) senders
Richard Shapiro       1328
Steven J. Kean         923
Jeffrey T. Hodge       658
Susan Bailey           579
Sara Shackleton        523
Steven Harris          488
Richard B. Sanders     448
Tana Jones             426
Carol St Clair         423
James D. Steffes       415
Name: name, dtype: int64


Keep exploring the dataset, which other questions would you ask?

Work in pairs. Give each other a challenge and try to solve it.